Preâmbulo

In [1]:
# Inteligência Artificial
# Alunos: Helon e Tiago

import pandas as pd # para manipulação e análise de dados
import numpy as np # computação científica - utilizada nas curvas gaussinas

import matplotlib.pyplot as plt # para geração de gráficos

from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, auc # para a matrix de confusão
from sklearn.model_selection import cross_val_score # para a validação cruzada

import joblib # usamos para salvar e carregar objetos Python, como modelos e variáveis ​​treinados, para/do disco

import os # para executar operações relacionadas ao sistema operacional, como criar diretórios, 
import json


In [2]:
# Bibliotecas específicas para o aprendizado de máquina
from sklearn.model_selection import train_test_split # permite dividir o conjunto de dados em TREINAMENTO E TESTE


from sklearn.neural_network import MLPClassifier # é utilizada para criar e treinar diferentes modelos de RNA com diversas combinações de hiperparâmetros

from sklearn.metrics import roc_auc_score # é utilizada para calcular a métrica de área sob a curva ROC (Receiver Operating Characteristic) que é frequentemente usada para avaliar o desempenho de modelos de classificação binária

# Importar a biblioteca para busca de hiperparâmetros com algoritmos genéticos


from sklearn.neural_network import MLPClassifier




Carregamento do Dataset

In [3]:
dataset = pd.read_csv("D:\\Users\\tiago\\OneDrive\\SI UFRRJ\\5p\\Inteligencia Artificial\\Trabalhos\\Trabalho 5 - Redes Neurais Art e Tratamento de Dados\\Fase 2\\Dados escolhidos (veio la da Fase 1)\\dados_escolhidos (copia Fase 1).csv", sep = ";")
# Substitua 'id' pelo nome correto da primeira coluna que você deseja remover
dataset = dataset.iloc[:, 1:] # removendo a primeira coluna: seleciona todas as linhas (indicadas pelo :) e todas as colunas a partir da segunda coluna (índice 1 em diante)

In [4]:
dataset.head()

,V1,V2,V7,V8,V12,V14,V15
0,0.192459,0.050191,0.397059,0.131057,0.005847,0.635945,1.0
1,0.303054,0.315094,0.779412,0.509340,0.007254,0.456221,1.0
2,0.002424,0.521228,0.602941,0.050738,0.005275,0.239631,1.0
3,0.014140,0.427120,0.794118,0.358853,0.005671,0.410138,1.0
4,0.260720,0.172144,0.838235,0.193550,0.007386,0.797235,1.0


In [5]:
# Dividindo em conjunto de treinamento e teste (30% dos dados será para teste -> test_size = 0.3)
V1_norm_treino, V1_norm_teste, V2_norm_treino, V2_norm_teste, V7_norm_treino, V7_norm_teste, V8_norm_treino, V8_norm_teste, V12_norm_treino, V12_norm_teste, V14_norm_treino, V14_norm_teste = train_test_split (dataset['V1'], dataset['V2'], dataset['V7'], dataset['V8'], dataset['V12'], dataset['V14'], test_size=0.3)


# Crie um DataFrame contendo todas as colunas de entrada
x_treino = pd.DataFrame({
    'V1': V1_norm_treino,
    'V2': V2_norm_treino,
    'V7': V7_norm_treino,
    'V8': V8_norm_treino,
    'V12': V12_norm_treino,
    'V14': V14_norm_treino
})

# Conjunto de teste (pra previsão)
x_teste = pd.DataFrame({
    'V1': V1_norm_teste,
    'V2': V2_norm_teste,
    'V7': V7_norm_teste,
    'V8': V8_norm_teste,
    'V12': V12_norm_teste,
    'V14': V14_norm_teste
})


# Y é a sáida, o resultado esperado. Que aqui, é a classe (-> a coluna 'V15' que representa as classes).
y_de_treino = dataset.loc[x_treino.index, 'V15']
y_de_teste = dataset.loc[x_teste.index, 'V15'] # Calcule a variável de saída real para o conjunto de teste (y_de_teste)

#EXPORTANDO variáveis para um arquivo Python:

# Aqui o caminho e nome do arquivo Python que vamos criar
arquivo_python = 'variaveis_mlp.py'

# Variáveis que vamos exportar
variaveis_exportar = [V1_norm_treino, V1_norm_teste, V2_norm_treino, V2_norm_teste, V7_norm_treino, V7_norm_teste, V8_norm_treino, V8_norm_teste, V12_norm_treino, V12_norm_teste, V14_norm_treino, V14_norm_teste, x_treino, x_teste, y_de_treino, y_de_teste]

# Salvar as variáveis no arquivo Python
joblib.dump(variaveis_exportar, arquivo_python)

['variaveis_mlp.py']

Achando melhores parâmetros iniciais - Fazendo combinações (Processamentos)

In [6]:
# No contexto de aprendizado de máquina, um "modelo de rede" refere-se a uma instância específica de um modelo de rede neural.


# A melhor função de ativação para a maioria dos casos não é a sigmóide ("logistic") MAS SIM a "reLU" !

# Função para treinar e avaliar a rede neural com hiperparâmetros específicos
def train_and_evaluate_model(hidden_layer_sizes, learning_rate_init, momentum):
    rna = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        max_iter=2000,
        tol=1e-7,
        learning_rate_init=learning_rate_init,
        momentum=momentum,
        solver="sgd",
        activation="relu",
        learning_rate="constant",
        verbose=0
    ) # <- Crie a instância do classificador MLP:
    # (nº de camadas escondidas (tupla: qntd neurônios camd 1, neur camd 2 ) , nº máx de iterações/épocas , tolerância da melhora limite do erro , estratégia = descida gdt estocástico , função de ativação, taxa constante )        - Rede neural artificial de multiplas camadas
    # verbose: para podermos ver o que está acontecendo em cada época
    # SGD significa Stochastic Gradient Descent = descida do gradiente estocástico (é um algoritmo de otimização) #EXTRA

    # Treinar o modelo com o conjunto de treinamento
    rna.fit(x_treino, y_de_treino)

    # Previsão do conjunto de teste (com as variáveis separadas para esse "teste" - "posterior") - Avaliar o modelo no conjunto de teste:
    y_pred = rna.predict(x_teste)
    accuracy = np.mean(y_pred == y_de_teste) * 100
    return accuracy, rna


In [7]:
# Valores para passar como "hiperparâmetros"
hidden_layer_sizes_values = [(10, 5), (20, 10), (30, 15)]
learning_rate_init_values = [0.01, 0.1, 0.2]
momentum_values = [0.5, 0.9, 0.99]

# Número de treinamentos para cada combinação de hiperparâmetros
num_de_treinos = 20 # 20 pelo menos que foi pedido

# Lista para armazenar as estatísticas de todos os modelos treinados
global estatisticas_dos_modelos
estatisticas_dos_modelos = []


In [8]:
# Lista para armazenar as estatísticas dos modelos treinados
estatisticas_dos_modelos = []

# Lista para armazenar todos os modelos treinados com diferentes hiperparâmetros
todos_os_modelos = []


# Loop para testar diferentes combinações de hiperparâmetros
for hidden_layer_sizes in hidden_layer_sizes_values:
    for learning_rate_init in learning_rate_init_values:
        for momentum in momentum_values:
            # Lista para armazenar as acurácias de cada treinamento
            accuracies = []


            # Cria o modelo com os hiperparâmetros específicos
            rna = MLPClassifier(
                hidden_layer_sizes=hidden_layer_sizes,
                max_iter=2000,
                tol=1e-7,
                learning_rate_init=learning_rate_init,
                momentum=momentum,
                solver="sgd",
                activation="relu",
                learning_rate="constant",
                verbose=0
            )

            # Executa o treinamento repetido (20 vezes)
            for _ in range(num_de_treinos):
                rna.fit(x_treino, y_de_treino)
                # Executa a validação cruzada e obtém as acurácias nos folds:
                # executa a validação cruzada (usando cross_val_score com cv=5) para cada combinação de hiperparâmetros, o que divide os dados em 5 conjuntos e realiza o treinamento e teste 5 vezes, retornando as acurácias médias.
                scores = cross_val_score(rna, x_treino, y_de_treino, cv=5, scoring='accuracy')
                accuracy = np.mean(scores)
                accuracies.append(accuracy)

                 # Armazena as informações do modelo atual
                modelo_info = {
                    "hidden_layer_sizes": hidden_layer_sizes,
                    "learning_rate_init": learning_rate_init,
                    "momentum": momentum,
                    "accuracy": accuracy,
                }
                #Adiciona as informações do modelo à lista de todos os modelos
                todos_os_modelos.append(modelo_info)

                #Portanto, no final do código, teremos uma lista de modelos treinados "trained_models",
                # onde cada elemento da lista é uma instância do modelo MLP treinado com uma combinação específica de hiperparâmetros.
                # Esses modelos podem ser salvos posteriormente e usados para fazer previsões em novos dados sem a necessidade de treiná-los novamente

                # Salvar o modelo treinado imediatamente após o treinamento
                nome_do_arquivo = f"model_{hidden_layer_sizes}_{learning_rate_init}_{momentum}_run_{len(todos_os_modelos)}_accuracy_{accuracy:.2f}.pkl"
                # Verificar se a pasta 'modelos' existe, se não existir, criar a pasta
                pasta_modelos = 'modelos'
                if not os.path.exists(pasta_modelos):
                    os.makedirs(pasta_modelos)
                # Caminho completo para salvar o modelo dentro da pasta 'modelos'
                modelo_salvar = os.path.join(pasta_modelos, nome_do_arquivo)
                joblib.dump(rna, modelo_salvar)


                #-\-\-\-\-\-\-\-\\--\-\-\-\-\\-\-

                # USO PARA FORMAS OS GRUPOS (POR HIPER PARAM. IGUAIS) - Armazena as estatísticas do modelo atual
                estatisticas_do_modelo = {
                    "hidden_layer_sizes": hidden_layer_sizes,
                    "learning_rate_init": learning_rate_init,
                    "momentum": momentum,
                    "accuracy": accuracies, #PLURAL?
                }

                # Salvar as estatísticas do modelo DENTRO DE UM ARQUIVO POR GRUPO (DE HIPERPARAMETRO IGUAIS) em um arquivo JSON - TOTALIZANDO 27 GRUPOS, 27JSON
                nome_do_arquivo_json = f"estatisticas_{hidden_layer_sizes}_{learning_rate_init}_{momentum}.json"
                pasta_estatistica_modelos_no_formato_json = 'estatistica_por_grupos_no_formato_json'
                if not os.path.exists(pasta_estatistica_modelos_no_formato_json):
                    os.makedirs(pasta_estatistica_modelos_no_formato_json)
                caminho_arquivo_json = os.path.join(pasta_estatistica_modelos_no_formato_json, nome_do_arquivo_json)
                with open(caminho_arquivo_json, 'w') as arquivo_json:
                    json.dump(estatisticas_do_modelo, arquivo_json)

            
            # Adicionar as estatísticas do modelo à lista geral
            estatisticas_dos_modelos.append(estatisticas_do_modelo)
            
            # Calcula as estatísticas das acurácias
            mean_accuracy = np.mean(accuracies)
            std_accuracy = np.std(accuracies)





# Imprimir as estatísticas dos modelos treinados
for stats in estatisticas_dos_modelos:
    print(stats)


d:\Users\tiago\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\Users\tiago\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\Users\tiago\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\Users\tiago\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the 

{'hidden_layer_sizes': (10, 5), 'learning_rate_init': 0.01, 'momentum': 0.5, 'accuracy': [0.6458498023715415, 0.6458498023715415, 0.6367588932806324, 0.6822134387351778, 0.6375494071146245, 0.7169960474308301, 0.6466403162055336, 0.6735177865612648, 0.6458498023715415, 0.6731225296442688, 0.7177865612648221, 0.6545454545454545, 0.7086956521739131, 0.6371541501976284, 0.6644268774703557, 0.6636363636363636, 0.6545454545454545, 0.6636363636363636, 0.7086956521739131, 0.6371541501976286]}
{'hidden_layer_sizes': (10, 5), 'learning_rate_init': 0.01, 'momentum': 0.9, 'accuracy': [0.7075098814229248, 0.6636363636363636, 0.7075098814229248, 0.6739130434782609, 0.7166007905138339, 0.7434782608695653, 0.7351778656126482, 0.6458498023715415, 0.7430830039525691, 0.6889328063241107, 0.6454545454545454, 0.7343873517786561, 0.7513833992094862, 0.6988142292490118, 0.7355731225296444, 0.7086956521739131, 0.7241106719367588, 0.716205533596838, 0.7169960474308301, 0.7347826086956522]}
{'hidden_layer_size

Pós-processamento final

In [9]:
# Ordenar os modelos pelas maiores acurácias primeiro.
todos_os_modelos.sort(key=lambda x: x["accuracy"], reverse=True)

# Imprimir as estatísticas dos modelos treinados
for i, modelo_info in enumerate(todos_os_modelos):
    print(f"Modelo {i+1}")
    print("Hiperparâmetros:", modelo_info["hidden_layer_sizes"], modelo_info["learning_rate_init"], modelo_info["momentum"])
    print("Acurácia:", modelo_info["accuracy"])
    print()


Modelo 1
Hiperparâmetros: (30, 15) 0.1 0.99
Acurácia: 0.7968379446640317

Modelo 2
Hiperparâmetros: (30, 15) 0.01 0.9
Acurácia: 0.7960474308300395

Modelo 3
Hiperparâmetros: (30, 15) 0.01 0.9
Acurácia: 0.7873517786561266

Modelo 4
Hiperparâmetros: (20, 10) 0.01 0.9
Acurácia: 0.7873517786561265

Modelo 5
Hiperparâmetros: (20, 10) 0.2 0.9
Acurácia: 0.7869565217391304

Modelo 6
Hiperparâmetros: (30, 15) 0.01 0.9
Acurácia: 0.7869565217391304

Modelo 7
Hiperparâmetros: (30, 15) 0.01 0.9
Acurácia: 0.7782608695652173

Modelo 8
Hiperparâmetros: (30, 15) 0.01 0.99
Acurácia: 0.7778656126482214

Modelo 9
Hiperparâmetros: (20, 10) 0.01 0.9
Acurácia: 0.7703557312252964

Modelo 10
Hiperparâmetros: (20, 10) 0.1 0.9
Acurácia: 0.7703557312252964

Modelo 11
Hiperparâmetros: (30, 15) 0.01 0.9
Acurácia: 0.7699604743083004

Modelo 12
Hiperparâmetros: (20, 10) 0.01 0.9
Acurácia: 0.7695652173913043

Modelo 13
Hiperparâmetros: (20, 10) 0.01 0.9
Acurácia: 0.7695652173913043

Modelo 14
Hiperparâmetros: (30, 15)

In [10]:
# Imprimir as estatísticas dos modelos treinados
for stats in estatisticas_dos_modelos:
    accuracy_mean = np.mean(stats["accuracy"])
    accuracy_std = np.std(stats["accuracy"])
    print(f"Hiperparâmetros: {stats['hidden_layer_sizes']}, {stats['learning_rate_init']}, {stats['momentum']}")
    print(f"Média de Acurácia: {accuracy_mean:.4f}")
    print(f"Desvio Padrão de Acurácia: {accuracy_std:.4f}")
    print()


#PRINTEI AGORA VOU EXPORTAR PARA UM ARQUIVO

# Definir o nome do arquivo de texto para exportar as estatísticas
arquivo_txt = "estatisticas_modelos.txt"

# Abrir o arquivo em modo de escrita
with open(arquivo_txt, "w") as arquivo:
    # Escrever as estatísticas dos modelos no arquivo
    for stats in estatisticas_dos_modelos:
        accuracy_mean = np.mean(stats["accuracy"])
        accuracy_std = np.std(stats["accuracy"])
        modelo_info = f"Hiperparâmetros: {stats['hidden_layer_sizes']}, {stats['learning_rate_init']}, {stats['momentum']}\n"
        media_acuracia = f"Média de Acurácia: {accuracy_mean:.4f}\n"
        desvio_padrao_acuracia = f"Desvio Padrão de Acurácia: {accuracy_std:.4f}\n"
        arquivo.write(modelo_info)
        arquivo.write(media_acuracia)
        arquivo.write(desvio_padrao_acuracia)
        arquivo.write("\n")

Hiperparâmetros: (10, 5), 0.01, 0.5
Média de Acurácia: 0.6657
Desvio Padrão de Acurácia: 0.0269

Hiperparâmetros: (10, 5), 0.01, 0.9
Média de Acurácia: 0.7096
Desvio Padrão de Acurácia: 0.0309

Hiperparâmetros: (10, 5), 0.01, 0.99
Média de Acurácia: 0.6426
Desvio Padrão de Acurácia: 0.0331

Hiperparâmetros: (10, 5), 0.1, 0.5
Média de Acurácia: 0.7035
Desvio Padrão de Acurácia: 0.0314

Hiperparâmetros: (10, 5), 0.1, 0.9
Média de Acurácia: 0.7052
Desvio Padrão de Acurácia: 0.0296

Hiperparâmetros: (10, 5), 0.1, 0.99
Média de Acurácia: 0.6644
Desvio Padrão de Acurácia: 0.0374

Hiperparâmetros: (10, 5), 0.2, 0.5
Média de Acurácia: 0.6650
Desvio Padrão de Acurácia: 0.0347

Hiperparâmetros: (10, 5), 0.2, 0.9
Média de Acurácia: 0.6805
Desvio Padrão de Acurácia: 0.0244

Hiperparâmetros: (10, 5), 0.2, 0.99
Média de Acurácia: 0.6574
Desvio Padrão de Acurácia: 0.0363

Hiperparâmetros: (20, 10), 0.01, 0.5
Média de Acurácia: 0.6936
Desvio Padrão de Acurácia: 0.0187

Hiperparâmetros: (20, 10), 0.01,

In [11]:
# Obter o modelo com a maior acurácia (primeiro lugar da lista ordenada)
melhor_modelo = todos_os_modelos[0]

# Treinar o modelo selecionado novamente com todos os dados de treinamento
melhor_modelo_treinado = MLPClassifier(
    hidden_layer_sizes=melhor_modelo["hidden_layer_sizes"],
    max_iter=2000,
    tol=1e-7,
    learning_rate_init=melhor_modelo["learning_rate_init"],
    momentum=melhor_modelo["momentum"],
    solver="sgd",
    activation="relu",
    learning_rate="constant",
    verbose=0
)

melhor_modelo_treinado.fit(x_treino, y_de_treino)

# Fazer previsões no conjunto de teste
y_pred = melhor_modelo_treinado.predict(x_teste)

# Calcular a matriz de confusão
matriz_confusao = confusion_matrix(y_de_teste, y_pred)

print("Matriz de Confusão:")
print(matriz_confusao)

Matriz de Confusão:
[[ 7  8]
 [ 9 25]]


In [12]:
#EXPORTANDO VARIÁVEL para um arquivo Python:
# Coloque aqui o caminho e nome do arquivo Python que você deseja criar
arquivo_python = 'estatisticas_modelos_e_lista_de_modelos_mlp.py'

# Salvar a lista de estatísticas no arquivo Python
joblib.dump(todos_os_modelos, arquivo_python)

['estatisticas_modelos_e_lista_de_modelos_mlp.py']